In [1]:
import os
import cv2
import rasterio
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from shapely import geometry, ops
from tqdm import tqdm
from skimage.transform import resize
import imageio

In [2]:
def compute_ndvi(image):
    red = image[0].astype(np.float32)
    nir = image[3].astype(np.float32)

    ndvi = np.clip((nir - red) / (nir + red + 1e-8), 0, 1)
    return ndvi

def load_tif(path_tif):
    # Load image
    tif = rasterio.open(path_tif)
    img = tif.read()
    tif.close()
    #
    image_bounds = list(tif.bounds)
    image_poly = geometry.Polygon.from_bounds(*image_bounds)

    return img, image_poly


In [3]:
filenames = sorted(os.listdir('./outputs/'))

images = []
for filename in tqdm(filenames):
    image, poly = load_tif('./outputs/'+filename)
    images.append(image.copy())

100%|██████████| 7/7 [00:00<00:00, 57.65it/s]


In [4]:
# save animation
height,width = image.shape[1:]


images_out = []
for i, im in enumerate(images):
    im_out_8bit = (255*np.clip(np.moveaxis(im,0,-1)/3000,0,1)).astype(np.uint8)
    im_out_8bit_resized = cv2.resize(im_out_8bit, (0, 0), fx=0.5, fy=0.5)
#     im_out_8bit_resized = resize(im_out_8bit, (height//2,width//2,3) , order=3).astype(np.uint8)
    cv2.putText(im_out_8bit_resized, filenames[i].split('.')[0], (20,620), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,2)
    images_out.append(im_out_8bit_resized)

imageio.mimsave('images.gif', images_out, loop=0, duration=900)


In [5]:
changemaps = []
changemaps.append(np.zeros_like(images[0][0]))
for i in range(1, len(images)):
    ndvi1 = compute_ndvi(images[0])
    ndvi2 = compute_ndvi(images[i])

    threshold_low = 0.3
    threshold_high = 0.95
    threshold_change = -0.15

    ndvi1[ndvi1 < threshold_low]  = np.nan  # old image: high values in savi
    ndvi2[ndvi2 > threshold_high] = np.nan  # new image: low values in savi
    changemap = ndvi2 - ndvi1
    changemap[changemap > threshold_change] = np.nan
    changemap = abs(np.nan_to_num(changemap))
    changemaps.append(changemap.copy())

In [6]:
changes_out = []
for i, im in enumerate(changemaps):
    im_out_8bit = (255*(cm.plasma(changemaps[i])[:,:,:3])).astype(np.uint8)
    im_out_8bit_resized = cv2.resize(im_out_8bit, (0, 0), fx=0.5, fy=0.5)
    cv2.putText(im_out_8bit_resized, filenames[i].split('.')[0], (20,620), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,2)
    changes_out.append(im_out_8bit_resized)

imageio.mimsave('changes.gif', changes_out, loop=0, duration=900)
